In [96]:
!pip install darts

In [97]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta, TCNModel
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, RepeatVector, TimeDistributed, Flatten
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D


In [98]:
import sys
import time
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


EPOCH = 300    
FC_N = 12               # forecast periods
FC_STRIDE = 10
FC_START = "19580801"   # period at which to split training and validation dataset
DROP = 0.1
LEARN = 0.001

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:

data = pd.read_csv("/content/On-street_Parking_Bay_Sensors (2).csv")

In [101]:
y = pd.get_dummies(data.status, prefix='status')


In [102]:
data_data = pd.concat([data, y], axis=1, join='inner')
display(data_data)

,bay_id,st_marker_id,status,location,lat,lon,status_Present,status_Unoccupied
0,1399,1581S,Unoccupied,"(-37.81865397291395, 144.96367840905828)",-37.818654,144.963678,0,1
1,2242,3930N,Unoccupied,"(-37.81219457073095, 144.9546289525064)",-37.812195,144.954629,0,1
2,6092,13251S,Unoccupied,"(-37.82011525242285, 144.9431517749976)",-37.820115,144.943152,0,1
3,9064,9278N,Unoccupied,"(-37.82897744884528, 144.969902966706)",-37.828977,144.969903,0,1
4,8979,7891W,Unoccupied,"(-37.825247665661855, 144.9707237360832)",-37.825248,144.970724,0,1
...,...,...,...,...,...,...,...,...
948,2643,10117W,Unoccupied,"(-37.80331033543291, 144.94795547514428)",-37.803310,144.947955,0,1
949,4422,C7684,Unoccupied,"(-37.80367821515115, 144.95521518472984)",-37.803678,144.955215,0,1
950,4086,C6286,Unoccupied,"(-37.80497529927409, 144.95078932967044)",-37.804975,144.950789,0,1
951,4069,C6176,Unoccupied,"(-37.80373641156282, 144.95251392706675)",-37.803736,144.952514,0,1


In [103]:
data_data.drop(columns=['status_Unoccupied'], axis = 1, inplace = True)

In [104]:
data_data.drop(columns=['status'], axis = 1, inplace = True)

In [105]:
data_data = data_data.rename(columns={'status_Present': 'status'})

In [106]:
data_data

,bay_id,st_marker_id,location,lat,lon,status
0,1399,1581S,"(-37.81865397291395, 144.96367840905828)",-37.818654,144.963678,0
1,2242,3930N,"(-37.81219457073095, 144.9546289525064)",-37.812195,144.954629,0
2,6092,13251S,"(-37.82011525242285, 144.9431517749976)",-37.820115,144.943152,0
3,9064,9278N,"(-37.82897744884528, 144.969902966706)",-37.828977,144.969903,0
4,8979,7891W,"(-37.825247665661855, 144.9707237360832)",-37.825248,144.970724,0
...,...,...,...,...,...,...
948,2643,10117W,"(-37.80331033543291, 144.94795547514428)",-37.803310,144.947955,0
949,4422,C7684,"(-37.80367821515115, 144.95521518472984)",-37.803678,144.955215,0
950,4086,C6286,"(-37.80497529927409, 144.95078932967044)",-37.804975,144.950789,0
951,4069,C6176,"(-37.80373641156282, 144.95251392706675)",-37.803736,144.952514,0


In [107]:
df = data_data.copy()

In [108]:
print('Total num of missing values:') 
print(df.status.isna().sum())
print('')

Total num of missing values:
0



In [109]:
df

,bay_id,st_marker_id,location,lat,lon,status
0,1399,1581S,"(-37.81865397291395, 144.96367840905828)",-37.818654,144.963678,0
1,2242,3930N,"(-37.81219457073095, 144.9546289525064)",-37.812195,144.954629,0
2,6092,13251S,"(-37.82011525242285, 144.9431517749976)",-37.820115,144.943152,0
3,9064,9278N,"(-37.82897744884528, 144.969902966706)",-37.828977,144.969903,0
4,8979,7891W,"(-37.825247665661855, 144.9707237360832)",-37.825248,144.970724,0
...,...,...,...,...,...,...
948,2643,10117W,"(-37.80331033543291, 144.94795547514428)",-37.803310,144.947955,0
949,4422,C7684,"(-37.80367821515115, 144.95521518472984)",-37.803678,144.955215,0
950,4086,C6286,"(-37.80497529927409, 144.95078932967044)",-37.804975,144.950789,0
951,4069,C6176,"(-37.80373641156282, 144.95251392706675)",-37.803736,144.952514,0


In [110]:
df = df.drop(['bay_id', 'lat', 'lon', 'st_marker_id', 'location'], axis = 1)

In [111]:
df

,status
0,0
1,0
2,0
3,0
4,0
...,...
948,0
949,0
950,0
951,0


In [112]:
# Split train data and test data
train_size = int(len(df)*0.7)

train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

In [113]:
scaler = MinMaxScaler().fit(train_data)
train_scaled = scaler.transform(train_data)
test_scaled = scaler.transform(test_data)

In [114]:
# Create input dataset
def create_dataset (X, look_back = 1):
    Xs, ys = [], []
 
    for i in range(len(X)-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])
 
    return np.array(Xs), np.array(ys)
LOOK_BACK = 30
X_train, y_train = create_dataset(train_scaled,LOOK_BACK)
X_test, y_test = create_dataset(test_scaled,LOOK_BACK)
# Print data shape
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape) 
print('y_test.shape:', y_test.shape)

X_train.shape: (637, 30, 1)
y_train.shape: (637, 1)
X_test.shape: (256, 30, 1)
y_test.shape: (256, 1)


In [115]:
    model_TCN = TCNModel(
        input_chunk_length=FC_N+1,
        output_chunk_length=FC_N, 
        n_epochs=EPOCH,
        dropout=DROP,
        dilation_base=2,
        weight_norm=True,
        batch_size=16,
        kernel_size=5,
        num_filters=3,
        optimizer_kwargs={'lr': LEARN},
        log_tensorboard=True,
        force_reset=True,
        random_state=42)

In [116]:
#from tensorflow.keras.optimizers import adam


In [117]:
model_tcn = Sequential()
model_tcn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_tcn.add(MaxPooling1D(pool_size=4))
model_tcn.add(Flatten())
model_tcn.add(Dense(100, activation='relu'))
model_tcn.add(Dense(1))
model_tcn.compile(loss='mse', optimizer= 'adam')
model_tcn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 29, 64)            192       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 7, 64)            0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 448)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               44900     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 45,193
Trainable params: 45,193
Non-trainable params: 0
__________________________________________________

In [118]:
tcn_history = model_tcn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs= 20, verbose=2)

Epoch 1/20
20/20 - 1s - loss: 0.0115 - val_loss: 0.0274 - 879ms/epoch - 44ms/step
Epoch 2/20
20/20 - 0s - loss: 0.0110 - val_loss: 0.0272 - 109ms/epoch - 5ms/step
Epoch 3/20
20/20 - 0s - loss: 0.0110 - val_loss: 0.0273 - 96ms/epoch - 5ms/step
Epoch 4/20
20/20 - 0s - loss: 0.0109 - val_loss: 0.0280 - 92ms/epoch - 5ms/step
Epoch 5/20
20/20 - 0s - loss: 0.0106 - val_loss: 0.0279 - 134ms/epoch - 7ms/step
Epoch 6/20
20/20 - 0s - loss: 0.0107 - val_loss: 0.0271 - 116ms/epoch - 6ms/step
Epoch 7/20
20/20 - 0s - loss: 0.0105 - val_loss: 0.0278 - 206ms/epoch - 10ms/step
Epoch 8/20
20/20 - 0s - loss: 0.0105 - val_loss: 0.0278 - 98ms/epoch - 5ms/step
Epoch 9/20
20/20 - 0s - loss: 0.0105 - val_loss: 0.0275 - 107ms/epoch - 5ms/step
Epoch 10/20
20/20 - 0s - loss: 0.0105 - val_loss: 0.0276 - 80ms/epoch - 4ms/step
Epoch 11/20
20/20 - 0s - loss: 0.0105 - val_loss: 0.0276 - 137ms/epoch - 7ms/step
Epoch 12/20
20/20 - 0s - loss: 0.0104 - val_loss: 0.0279 - 86ms/epoch - 4ms/step
Epoch 13/20
20/20 - 0s - los

In [119]:
### Lets Do the prediction and check performance metrics
train_predict=model_tcn.predict(X_train)
test_predict=model_tcn.predict(X_test)

In [120]:
test_predict

array([[ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0.01133057],
       [ 0

In [121]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)
test_predict[test_predict > 10.0] = 1
test_predict[test_predict < 10.0] = 0


In [122]:
from sklearn import metrics

In [123]:
print(metrics.mean_absolute_error(y_test, test_predict))

0.02734375


In [124]:
from sklearn.metrics import accuracy_score

In [125]:
accuracy_score(y_test, test_predict)

0.97265625

In [126]:

print(np.sqrt(metrics.mean_squared_error(y_test, test_predict)))

0.16535945694153692


In [127]:
print(metrics.mean_squared_error(y_test, test_predict))

0.02734375
